In [0]:
%python
bronze_table = spark.table("plstocks.bronze_financial_reports")
display(bronze_table)

In [0]:
%python
from pyspark.sql.functions import col, regexp_replace, length

# Usuń rekordy zawierające litery i zamień kolumny na int (oprócz "ticker")
columns_to_convert = [column for column in bronze_table.columns if column != "ticker"]

bronze_table_cleaned = bronze_table
for column in columns_to_convert:
    bronze_table_cleaned = bronze_table_cleaned.withColumn(
        column, 
        regexp_replace(col(column), '[^0-9]', '').cast('int')
    )

# Usuń wiersze, które mają null po konwersji na int
bronze_table_cleaned = bronze_table_cleaned.na.drop()

# Filtruj wiersze, gdzie liczba znaków w kolumnie "year" jest równa 4
bronze_table_cleaned = bronze_table_cleaned.filter(length(col("year")) == 4)

# Usuń wiersze z brakującymi wartościami w kluczowych kolumnach
silver_table = bronze_table_cleaned.dropna(subset=["ticker", "year", "ebitda"])

# Filtruj wiersze, gdzie wartość EBITDA jest większa niż 0
silver_table = silver_table.filter(col("ebitda") > 0)

# Usuń duplikaty na podstawie kolumny "ticker" i "year"
silver_table = silver_table.dropDuplicates(["ticker", "year"])

# Filtruj wiersze, gdzie rok jest w rozsądnym zakresie (np. 1900-2100)
silver_table = silver_table.filter((col("year") >= 1900) & (col("year") <= 2100))

# Sprawdź, czy kolumna "ticker" zawiera tylko litery i cyfry
silver_table = silver_table.filter(col("ticker").rlike("^[a-zA-Z0-9]+$"))

# Filtruj wiersze, gdzie EBITDA jest większa niż EBIT
silver_table = silver_table.filter(col("ebitda") >= col("ebit"))

# Wyświetl przetworzoną tabelę Silver
display(silver_table)

In [0]:
%python
from delta.tables import DeltaTable

if DeltaTable.isDeltaTable(spark, "plstocks.silver_financial_reports"):
    deltaTable = DeltaTable.forPath(spark, "plstocks.silver_financial_reports")
    
    deltaTable.alias("existing") \
        .merge(
            silver_table.alias("new"),
            "existing.ticker = new.ticker AND existing.year = new.year"
        ) \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()
else:
    silver_table.write.format("delta").mode("overwrite").saveAsTable("plstocks.silver_financial_reports")